## Chapter 8
# Writing generic functions

## Exercises

**8-0.** Compile, execute and test the programs in this chapter.

Implemented below:

In [1]:
#include <algorithm>
#include <stdexcept>
#include <vector>

In [2]:
using std::domain_error;

In [3]:
using std::vector;

In [4]:
using std::sort;

In [5]:
template <class T> T median(vector<T> v) {
    typedef typename vector<T>::size_type vec_sz;
    
    vec_sz size = v.size();
    if (size == 0) {
        throw domain_error("median of an empty vector");
    }
    sort(v.begin(), v.end());

    vec_sz mid = size / 2;
    return size % 2 == 0 ? (v[mid] + v[mid - 1]) / 2 : v[mid];
}

In [6]:
vector<int> ints;
ints.push_back(4);
ints.push_back(3);
ints.push_back(1);
ints.push_back(1);
ints.push_back(2);
ints.push_back(3);
median(ints);

(int) 2


In [7]:
vector<double> doubles;
doubles.push_back(4);
doubles.push_back(3);
doubles.push_back(1);
doubles.push_back(1);
doubles.push_back(2);
doubles.push_back(3);
median(doubles);

(double) 2.5000000


In [8]:
// using `my_` prefix to avoid ambiguity with <algorithm> library
template <class In, class X> In my_find(In begin, In end, const X& x) {
    while (begin != end && *begin != x) {
        ++begin;
    }
    return begin;
}

In [9]:
*my_find(doubles.begin(), doubles.end(), 2);

(double) 2.0000000


In [10]:
template <class In, class Out> Out my_copy(In begin, In end, Out dest) {
    while (begin != end) {
        *dest++ = *begin++;
    }
    return dest;
}

In [11]:
#include <iterator>
using std::back_inserter;

vector<double> doubles_copy;

my_copy(doubles.begin(), doubles.end(), back_inserter(doubles_copy));

doubles_copy;

(std::vector<double> &) { 4.0000000, 3.0000000, 1.0000000, 1.0000000, 2.0000000, 3.0000000 }


In [12]:
template<class For, class X> void my_replace (For beg, For end, const X& x, const X& y) {
    while (beg != end) {
        if (*beg == x) {
            *beg = y;
        }
        ++beg;
    }
}

In [13]:
my_replace(doubles_copy.begin(), doubles_copy.end(), 1.0, 5.0);

doubles_copy;

(std::vector<double> &) { 4.0000000, 3.0000000, 5.0000000, 5.0000000, 2.0000000, 3.0000000 }


In [14]:
template<class Element> void swap(Element& a, Element& b) {
    Element tmp = a;
    a = b;
    b = tmp;
}

template <class Bi> void my_reverse(Bi begin, Bi end) {
    while (begin != end) {
        --end;
        if (begin != end) {
            swap(*begin++, *end);
        }
    }
}

In [15]:
my_reverse(doubles_copy.begin(), doubles_copy.end());
doubles_copy;

(std::vector<double> &) { 3.0000000, 2.0000000, 5.0000000, 5.0000000, 3.0000000, 4.0000000 }


In [16]:
template <class Ran, class X> bool my_binary_search(Ran begin, Ran end, const X& x) {
    while (begin < end) {
        Ran mid = begin + (end - begin) / 2;
        if (x < *mid)
            end = mid;
        else if (*mid < x)
            begin = mid + 1;
        else
            return true;
    }
    return false;
}

In [17]:
sort(doubles_copy.begin(), doubles_copy.end());
my_binary_search(doubles_copy.begin(), doubles_copy.end(), 3.0);

(bool) true


In [18]:
#include <cctype>
#include <string>

using std::isspace;

In [19]:
using std::find_if;

In [20]:
using std::string;

In [21]:
bool not_space(const char& ch) {
    return !isspace(ch);
}

In [22]:
template <class Out> void split (const string& str, Out os) {
    typedef string::const_iterator iter;
    
    iter i = str.begin();
    while (i != str.end()) {
        i = find_if(i, str.end(), not_space);
        
        iter j = find_if(i, str.end(), isspace);
        
        if (i != str.end()) {
            *os++ = string(i, j);
        }
        
        i = j;
    }
}

In [23]:
#include <iostream>

using std::ostream_iterator;

In [24]:
using std::cout;

In [25]:
split("home is where I want to be, pick me up and turn me round", ostream_iterator<string>(cout, "\n"));

home
is
where
I
want
to
be,
pick
me
up
and
turn
me
round


(void) @0x700010390ea0


**8-1.** Note that the various `analysis` functions we wrote in 6.2/110 share the same behavior; they differ only in terms of the functions they call to calculate the final grade. Write a template function, parameterized by the type of the grading function, and use that function to evaluate the grading schemes.

Implemented in `analysis_template.cpp`:
```
$ gcc analysis_template.cpp ../chapter_6/grade.cpp ../chapter_6/Student_info.cpp -o analysis_template.out -lstdc++
$ ./analysis_template.out
Karl 80 90 90 0 89
Jark 78 95 80 90 80
median: median(did) = 85.6, median(didnt) = 87.6
average: median(did) = 86.9333, median(didnt) = 75.8667
median of homework turned in: median(did) = 85.6, median(didnt) = 87.8
```

(Here is the function definition: `template <class GradingScheme> double template_analysis(const vector<Student_info>& students, GradingScheme grading_scheme`)

**8-2.** Implement the following library algorithms, which we used in Chapter 6 and described in 6.5/121. Specify what kinds of iterators they require. Try to minimize the number of distinct iterator operations that each function requires. After you have finished your implementation, see B3/322 to see how well you did.

```
equal(b, e, b2)
find(b, e, t)
copy(b, e, d)
remove_copy_if(b, e, d, p)
transform(b, e, d, f)
accumulate(b, e, t)
search(b, e, b2, e2)
find_if(b, e, p)
remove_copy(b, e, d, t)
remove(b, e, t)
partition(b, e, p)
```


In [26]:
template<class In1, class In2> bool my_equal(In1 begin, In1 end, In2 begin2) {
    while (begin != end) {
        if (!(*begin++ == *begin2++)) {
            return false;
        }
    }
    return true;
}

In [27]:
doubles;

(std::vector<double> &) { 4.0000000, 3.0000000, 1.0000000, 1.0000000, 2.0000000, 3.0000000 }


In [28]:
doubles_copy;

(std::vector<double> &) { 2.0000000, 3.0000000, 3.0000000, 4.0000000, 5.0000000, 5.0000000 }


In [29]:
my_equal(doubles_copy.begin(), doubles_copy.begin() + 2, doubles.end() - 2);

(bool) true


In [30]:
my_equal(doubles_copy.begin(), doubles_copy.begin() + 3, doubles.end() - 3);

(bool) false


`my_find` and `my_copy` already implemented above.

In [31]:
template <class In, class Out, class Pred> Out my_remove_copy_if(In begin, In end, Out dest, Pred predicate) {
    while (begin != end) {
        if (!predicate(*begin)) {
            *dest++ = *begin;
        }
        ++begin;
    }
    return dest;
}

In [32]:
#include <cctype>

// std::isspace returns int, not bool
bool space(const char& c) {
    return std::isspace(c);
}

string str = "Text with some   spaces";
my_remove_copy_if(str.begin(), str.end(), ostream_iterator<char>(cout), space);
cout << '\n';

Textwithsomespaces


(std::basic_ostream<char, std::char_traits<char> > &) @0x7fffc29da660


In [33]:
template <class In, class Out, class Func> void my_transform(In begin, In end, Out dest, Func f) {
    while (begin != end) {
        *dest++ = f(*begin++);
    }
}

In [34]:
doubles;

(std::vector<double> &) { 4.0000000, 3.0000000, 1.0000000, 1.0000000, 2.0000000, 3.0000000 }


In [35]:
double times_2(const double d) {
    return d * 2;
}

my_transform(doubles.begin(), doubles.end(), doubles_copy.begin(), times_2);
doubles_copy;

(std::vector<double> &) { 8.0000000, 6.0000000, 2.0000000, 2.0000000, 4.0000000, 6.0000000 }


In [36]:
template <class In, class T> T my_accumulate(In begin, In end, T t) {
    while (begin != end) {
        t += *begin++;
    }
    return t;
}

In [37]:
my_accumulate(doubles_copy.begin(), doubles_copy.end(), 0);

(int) 28


In [38]:
template <class In1, class In2> In1 my_search(In1 begin1, In1 end1, In2 begin2, In2 end2) {
    In1 match_start = end1;

    while (begin1 != end1 && begin2 != end2) {
        if (*begin1++ == *begin2) {
            match_start = begin1 - 1;
            ++begin2;
        } else if (match_start != end1) {
            return end1; // only matches part of the range.
        }
    }
    return match_start;
}

In [39]:
vector<double> doubles_section;
my_copy(doubles.begin() + 2, doubles.begin() + 4, back_inserter(doubles_section));
doubles_section;

(std::vector<double> &) { 1.0000000, 1.0000000 }


In [40]:
doubles;

(std::vector<double> &) { 4.0000000, 3.0000000, 1.0000000, 1.0000000, 2.0000000, 3.0000000 }


In [41]:
*my_search(doubles.begin(), doubles.end(), doubles_section.begin(), doubles_section.end());

(double) 1.0000000


In [42]:
template <class In, class Pred> In my_find_if(In begin, In end, const Pred predicate) {
    while (begin != end && !predicate(*begin)) {
        ++begin;
    }
    return begin;
}

In [43]:
bool equals_4(const double d) {
    return d == 4.0;
}

*my_find_if(doubles.begin(), doubles.end(), equals_4);

(double) 4.0000000


In [44]:
template <class In, class Out, class X> Out my_remove_copy(In begin, In end, Out dest, const X& x) {
    while (begin != end) {
        if (*begin != x) {
            *dest++ = *begin;
        }
        ++begin;
    }
    return dest;
}

In [45]:
my_remove_copy(str.begin(), str.end(), ostream_iterator<char>(cout), ' ');
cout << '\n';

Textwithsomespaces


(std::basic_ostream<char, std::char_traits<char> > &) @0x7fffc29da660


In [46]:
template <class In, class X> In my_remove(In begin, In end, const X& x) {
    In ret = begin;
    while (begin != end) {
        while (*begin == x) {
            ++begin;
        }
        *ret++ = *begin++;
    }

    return ret;
}

In [47]:
doubles;

(std::vector<double> &) { 4.0000000, 3.0000000, 1.0000000, 1.0000000, 2.0000000, 3.0000000 }


In [48]:
my_remove(doubles.begin(), doubles.end(), 1.0);
doubles;

(std::vector<double> &) { 4.0000000, 3.0000000, 2.0000000, 3.0000000, 2.0000000, 3.0000000 }


In [49]:
template <class In, class Func> In my_partition(In begin, In end, Func predicate) {
    while (begin != end) {
        while (predicate(*begin)) {
            ++begin;
            if (begin == end) {
                return begin;
            }
        }
        do {
            --end;
            if (begin == end) {
                return begin;
            }
        } while (!predicate(*end));

        swap (*begin++, *end);
    }
    
    return begin;
}

In [50]:
bool less_than_6(const double d) {
    return d < 6.0;
}

In [51]:
doubles_copy;

(std::vector<double> &) { 8.0000000, 6.0000000, 2.0000000, 2.0000000, 4.0000000, 6.0000000 }


In [52]:
my_partition(doubles_copy.begin(), doubles_copy.end(), less_than_6);

(std::__wrap_iter<double *>) @0x7f94f87fb020


In [53]:
doubles_copy;

(std::vector<double> &) { 4.0000000, 2.0000000, 2.0000000, 6.0000000, 8.0000000, 6.0000000 }


**8-3.** As we learned in 4.14/58, it can be expensive to return (or pass) a container by value. Yet the `median` function that we wrote in 8.1.1/140 passes the `vector` by value. Could we rewrite the `median` function to operate on iterators instead of passing the `vector`? If we did so, what would you expect the performance impact to be?

Yes, we could rewrite the `median` function to operate on iterators instead of passing the `vector`.  The iterators passed in would need to support random access.  The performance would be better since it would not entail a copy, but the improvement would be minimal, especially for large input size, since most of the time is spent in `sort`, which we still need to keep.

Here's an example:

In [54]:
template <class T, class Rand> T median_with_iterators(Rand begin, Rand end) {
    if (begin == end) {
        throw domain_error("median of an empty range");
    }
    sort(begin, end);

    Rand mid = begin + (end - begin) / 2;
    return (end - begin) % 2 == 0 ? (*mid + *(mid - 1)) / 2 : *mid;
}

In [55]:
median_with_iterators<double>(doubles_copy.begin(), doubles_copy.end());

(double) 5.0000000


**8-4.** Implement the `swap` function that we used in 8.2.5/148. Why did we call `swap` rather than exchange the values of `*beg` and `*end` directly? _Hint:_ Try it and see.

(Implemented above for use in `my_reverse`.)

In [56]:
template <class Element, class Bi> void my_reverse_with_inline_swap_attempt(Bi begin, Bi end) {
    while (begin != end) {
        --end;
        if (begin != end) {
            Element tmp = *begin;
            *begin++ = *end;
            *end = tmp;
        }
    }
}

In [57]:
doubles_copy;

(std::vector<double> &) { 2.0000000, 2.0000000, 4.0000000, 6.0000000, 6.0000000, 8.0000000 }


In [58]:
my_reverse_with_inline_swap_attempt<double>(doubles_copy.begin(), doubles_copy.end());
doubles_copy;

(std::vector<double> &) { 8.0000000, 6.0000000, 6.0000000, 4.0000000, 2.0000000, 2.0000000 }


As the method above shows, if we do the swapping inline, we need to name and instantiate a template class `Element`, whereas delegating to a function allows us to be ignorant of the inner type completely.  This also requires that the caller specify a template type (`<double>`) explicitly.

Taking the question more literally, we wouldn't "exchange the values of `*beg` and `*end` directly" without a tmp variable since we would simply overwrite one of them and lose access to it.  This is independent of the question of inlining, though.

**8-5.** Reimplement the `gen_sentence` and `xref` functions from Chapter 7 to use the output iterators rather than putting their entire output in one data structure. Test these new versions by writing programs that attach the output iterator directly to the standard output, and by storing the results in a `list<string>` and a `map<string, vector<int> >`, respectively.

Implemented in `chapter_7/sentence_generator_test.cpp`.

For the `map<string, vector<int> >` case, I think the problem is ill-defined.  Using a back-inserter on the map won't work in the case where the value is a vector that needs to be retrieved at arbitrary points in the looping code.  It would work for finding the _first_ or _last_ line, but in order to have the same behavior, the code would need to still return a map from `string` to some container of `int`s.

**8-6.** Suppose that `m` has type `map<int, string>`, and that we encounter a call to `copy(m.begin(), m.end(), back_inserter(x))`. What can we say about the type of `x`? What if the call were `copy(x.begin(), x.end(), back_inserter(m))` instead?

If the code is correct, then `x` must have a container that holds `pair<int, string>` elements, such as a `map<int, string>` or a `vector<pair<int, string> >`.  The call `copy(x.begin(), x.end(), back_inserter(m))` would not work, since `map` has no method `push_back`. We would need somethine like `std::inserter(m, m.end())` instead. Examples:

In [60]:
#include <map>

In [61]:
std::map<int, string> m;
vector<std::pair<int, string> > x;

m[1] = "hello";
m[2] = "goodbye";
copy(m.begin(), m.end(), back_inserter(x));
x;

(std::vector<std::pair<int, std::string> > &) { {1 , "hello"}, {2 , "goodbye"} }


In [62]:
x.push_back(std::pair<int, string>(3, "another one"));
copy(x.begin(), x.end(), back_inserter(m)); // fails!

In file included from input_line_3:1:
In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/include/c++/v1/algorithm:640:
In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/include/c++/v1/memory:635:
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/include/c++/v1/iterator:775:21: error: no member named 'push_back' in 'std::__1::map<int, std::__1::basic_string<char>, std::__1::less<int>,
      std::__1::allocator<std::__1::pair<const int, std::__1::basic_string<char> > > >'
        {container->push_back(_VSTD::move(__value_)); return *this;}
         ~~~~~~~~~  ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/include/c++/v1/algorithm:1706:19: note: in instantiation of member function 'std::__1::back_insert_iterator<std::__1::map<int, std::__1::basic_string<char>,
      std::__1::less<int>, std::__1::allocator<std::

ename: evalue

**8-7.** Why doesn't the `max` function use two template parameters, one for each argument type?

If the `max` function used two template parameters, it could not guarantee that the comparison operator would behave as expected across both types.

**8-8.** In the `binary_search` function in 8.26/148, why didn't we write `(begin + end) / 2` instead of the more complicated `begin + (end - begin) / 2`?

The type that the expression `(begin + end) / 2` evaluates to is an `int`, whereas the expression `begin + (end - begin) / 2` evaluates to an iterator. (`it + (it - it) / 2 => it + it / 2 => it + int => it`)